In [1]:
from __future__ import print_function
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as image
plt.style.use("ggplot")

from skimage import io

from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
import ipywidgets as widgets
import time

In [2]:
def update(assign,img_data,k):
    centroids=np.zeros((k,3))
    cluster_size=np.zeros(k)
    for i in range(img_data.shape[0]):
        cluster=int(assign[i])
        centroids[cluster]+=img_data[i]
        cluster_size[cluster]+=1
    for i in range(k):
        if(cluster_size[i]==0):
            continue
        centroids[i]/=cluster_size[i]
    return centroids    

In [3]:
def assign(centroids,k,img_data,assign_prev,mini_distance):
    assig=np.zeros(img_data.shape[0])
    mini_distance1=np.zeros(img_data.shape[0])
    for i in range(img_data.shape[0]):
        if(dist(img_data[i],centroids[int(assign_prev[i])])<mini_distance[i]):
            assig[i]=assign_prev[i]
            mini_distance1[i]=dist(img_data[i],centroids[int(assign_prev[i])])
        
        else:
            mini= 1000
            for j in range(k):
                if (dist(img_data[i],centroids[j]) < mini):
                    mini=dist(img_data[i],centroids[j])
                    mini_distance1[i]=mini
                    assig[i]=j             

    return assig,mini_distance1          

In [4]:
def init_centroids(k):
    centroids=np.zeros((k,3))
    for i in range(k):
        centroids[i]=np.array([np.random.rand(),np.random.rand(),np.random.rand()])
    return centroids    

In [5]:
def dist(a,b):
    di=np.square(a[0]-b[0])+np.square(a[1]-b[1])+np.square(a[2]-b[2])
    return np.sqrt(di)

In [6]:
img_dir = 'images/'
@interact
def compression(image=os.listdir(img_dir), k=IntSlider(min=1,max=256,step=1,value=16,
                                                             continuous_update=False,
                                                             layout=dict(width='100%'))):
    input_img = io.imread(img_dir + image)
    start=time.time()
    img_data = (input_img / 255.0).reshape(-1, 3)
    
    centroids=init_centroids(k)
    
    assign1=np.zeros((img_data.shape[0]))
    assign_prev=np.zeros((img_data.shape[0]))
    mini_distance=np.zeros((img_data.shape[0]))
    
    iterations=5
    
    while iterations!=0:
        assign1,mini_distance=assign(centroids,k,img_data,assign_prev,mini_distance)
        centroids=update(assign1,img_data,k)
        assign_prev=assign1
        iterations-=1
   
    k_colors=np.zeros(img_data.shape)
    
    for i in range(img_data.shape[0]):
        k_colors[i]=centroids[int(assign1[i])]
        
    k_img = np.reshape(k_colors, (input_img.shape))
    
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18.5,10.5, forward=True)
    fig.suptitle('K-means Image Compression', fontsize=20)
    ax1.set_title('Compressed')
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.imshow(k_img)
    
    ax2.set_title('Original (16,777,216 colors)')
    ax2.set_xticks([])
    ax2.set_yticks([])
    ax2.imshow(input_img)
    
    plt.subplots_adjust(top=0.85)
    plt.show()
    end=time.time()
    print('Time taken by program was {} seconds'.format(end-start))

interactive(children=(Dropdown(description='image', options=('0-cat.jpg', '1-Dog.jpeg', '2-scenery.jpg', '3-Co…